Import mario modules

In [1]:
#! /usr/bin/python3
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import *

Import plotting modules

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
%matplotlib inline
%matplotlib notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Import TensorFlow

In [3]:
import tensorflow as tf

Setup plotting

In [4]:
%%capture
fig = plt.figure()
images = []

Setup enviroment

In [5]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, RIGHT_ONLY)
done = True

In [6]:
t = 0
def print_info(info, reward):
    global t 
    t += 1
    if not t % 100:
        print(info, reward)

In [7]:
save_animation = 1 # Saving animation increases running time

In [8]:
def handle_frame():
    global images
    if save_animation:
        image = plt.imshow(env.render(mode='rgb_array'))
        images.append([image])
    else:
        env.render()

In [9]:
def display_animation():
    anim = animation.ArtistAnimation(fig, images, interval=15, blit=True)
    rc('animation', html='jshtml')
    return(anim)

In [10]:
for step in range(50):
    if done:
        state = env.reset()
    action = env.action_space.sample()
    state, reward, done, info = env.step(env.action_space.sample())
    print_info(info, reward)
    handle_frame()

Animate results

In [11]:
if save_animation:
    display_animation()
    env.close()

In [12]:
print(SIMPLE_MOVEMENT)
print(env)

[['NOOP'], ['right'], ['right', 'A'], ['right', 'B'], ['right', 'A', 'B'], ['A'], ['left']]
<JoypadSpace<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>
